In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('test_data/lw.db')

/tmp/ipykernel_60222/1207926747.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
# Only selects first row of each group 
df = pd.read_sql_query("""
    select *, count("Freq [GHz]"), min(prediction), max(prediction),
    dense_rank() OVER(order BY "cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]") AS group_name
    from geometries
    where "Freq [GHz]" > ?
    and "Freq [GHz]" < ?
    group by "cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]"
    having max(prediction) < ?
""", con=conn, params=(12, 16, -10, ))

In [6]:
# Try to show all frequencies and predictions for each group  
df = pd.read_sql_query("""
    select * from geometries
    where ("cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]") in (
        select "cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]"
        from geometries
        where "Freq [GHz]" > ?
        and "Freq [GHz]" < ?
        group by "cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]"
        having max(prediction) < ?
    )
""", con=conn, params=(12, 16, -10, ))

In [7]:
df

,cpw_in [mm],feed_l [mm],patch_l [mm],cpw_g [mm],Feed_W [mm],ground_w [mm],patch_ground_w [mm],patch_w [mm],Freq [GHz],source_simulated,prediction
0,1.75,3.50,4.00,0.18,1.25,1.50,1.5,4.5,11.00,0,-3.664200
1,1.75,3.50,4.00,0.18,1.25,1.50,1.5,4.5,11.09,0,-3.818860
2,1.75,3.50,4.00,0.18,1.25,1.50,1.5,4.5,11.18,0,-4.088357
3,1.75,3.50,4.00,0.18,1.25,1.50,1.5,4.5,11.27,0,-4.528636
4,1.75,3.50,4.00,0.18,1.25,1.50,1.5,4.5,11.36,0,-4.984545
...,...,...,...,...,...,...,...,...,...,...,...
26255,2.30,4.25,3.75,0.72,0.75,0.75,1.0,5.0,19.64,0,-10.870723
26256,2.30,4.25,3.75,0.72,0.75,0.75,1.0,5.0,19.73,0,-11.501857
26257,2.30,4.25,3.75,0.72,0.75,0.75,1.0,5.0,19.82,0,-11.672318
26258,2.30,4.25,3.75,0.72,0.75,0.75,1.0,5.0,19.91,0,-11.555550


In [4]:
df.iloc[0]['performances']

'12.08,12.17,12.26,12.35,12.44,12.53,12.62,12.71,12.8,12.89,12.98,13.07,13.16,13.25,13.34,13.43,13.52,13.61,13.7,13.79,13.88,13.97,14.06,14.15,14.24,14.33,14.42,14.51,14.6,14.69,14.78,14.87,14.96,15.05,15.14,15.23,15.32,15.41,15.5,15.59,15.68,15.77,15.86,15.95,12.08,12.17,12.26,12.35,12.44,12.53,12.62,12.71,12.8,12.89,12.98,13.07,13.16,13.25,13.34,13.43,13.52,13.61,13.7,13.79,13.88,13.97,14.06,14.15,14.24,14.33,14.42,14.51,14.6,14.69,14.78,14.87,14.96,15.05,15.14,15.23,15.32,15.41,15.5,15.59,15.68,15.77,15.86,15.95,12.08,12.17,12.26,12.35,12.44,12.53,12.62,12.71,12.8,12.89,12.98,13.07,13.16,13.25,13.34,13.43,13.52,13.61,13.7,13.79,13.88,13.97,14.06,14.15,14.24,14.33,14.42,14.51,14.6,14.69,14.78,14.87,14.96,15.05,15.14,15.23,15.32,15.41,15.5,15.59,15.68,15.77,15.86,15.95,12.08,12.17,12.26,12.35,12.44,12.53,12.62,12.71,12.8,12.89,12.98,13.07,13.16,13.25,13.34,13.43,13.52,13.61,13.7,13.79,13.88,13.97,14.06,14.15,14.24,14.33,14.42,14.51,14.6,14.69,14.78,14.87,14.96,15.05,15.14,15.23,15.32,

In [5]:
df2 = df[['performances']]

In [6]:
df2.iloc[0]['performances']

'12.08,12.17,12.26,12.35,12.44,12.53,12.62,12.71,12.8,12.89,12.98,13.07,13.16,13.25,13.34,13.43,13.52,13.61,13.7,13.79,13.88,13.97,14.06,14.15,14.24,14.33,14.42,14.51,14.6,14.69,14.78,14.87,14.96,15.05,15.14,15.23,15.32,15.41,15.5,15.59,15.68,15.77,15.86,15.95,12.08,12.17,12.26,12.35,12.44,12.53,12.62,12.71,12.8,12.89,12.98,13.07,13.16,13.25,13.34,13.43,13.52,13.61,13.7,13.79,13.88,13.97,14.06,14.15,14.24,14.33,14.42,14.51,14.6,14.69,14.78,14.87,14.96,15.05,15.14,15.23,15.32,15.41,15.5,15.59,15.68,15.77,15.86,15.95,12.08,12.17,12.26,12.35,12.44,12.53,12.62,12.71,12.8,12.89,12.98,13.07,13.16,13.25,13.34,13.43,13.52,13.61,13.7,13.79,13.88,13.97,14.06,14.15,14.24,14.33,14.42,14.51,14.6,14.69,14.78,14.87,14.96,15.05,15.14,15.23,15.32,15.41,15.5,15.59,15.68,15.77,15.86,15.95,12.08,12.17,12.26,12.35,12.44,12.53,12.62,12.71,12.8,12.89,12.98,13.07,13.16,13.25,13.34,13.43,13.52,13.61,13.7,13.79,13.88,13.97,14.06,14.15,14.24,14.33,14.42,14.51,14.6,14.69,14.78,14.87,14.96,15.05,15.14,15.23,15.32,

In [7]:
df2[['Freq [GHz]', 'prediction']] = df2['performances'].str.split(';', expand=True)



/tmp/ipykernel_78366/2519720628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['Freq [GHz]', 'prediction']] = df2['performances'].str.split(';', expand=True)
/tmp/ipykernel_78366/2519720628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['Freq [GHz]', 'prediction']] = df2['performances'].str.split(';', expand=True)


In [8]:
df_freq = df2['Freq [GHz]'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Freq [GHz]')
df_pred = df2['prediction'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('prediction')


In [9]:
df_freq

0     12.08
0     12.17
0     12.26
0     12.35
0     12.44
      ...  
25    15.59
25    15.68
25    15.77
25    15.86
25    15.95
Name: Freq [GHz], Length: 11440, dtype: object

In [19]:
dfc = pd.DataFrame()
dfc['Freq [GHz]'] = df_freq
dfc['prediction'] = df_pred

In [23]:
df3 = df.drop(columns=["Freq [GHz]", "prediction", "performances"])

In [1]:
df_fin

NameError: name 'df_fin' is not defined

In [46]:
import numpy as np
df3['Geometry Name'] = range(0, len(df3))
df3['Geometry Name']
df_fin = pd.DataFrame(np.repeat(df3, 440, axis=0), columns=df3.columns)
df_fin["Freq [GHz]"] = df_freq.values
df_fin["prediction"] = df_pred.values

In [47]:
df_fin

,cpw_in [mm],feed_l [mm],patch_l [mm],cpw_g [mm],Feed_W [mm],ground_w [mm],patch_ground_w [mm],patch_w [mm],source_simulated,Geometry Name,Freq [GHz],prediction
0,1.75,3.50,4.00,0.30,1.25,1.50,1.5,4.5,0.0,0.0,12.08,-11.8380756378174
1,1.75,3.50,4.00,0.30,1.25,1.50,1.5,4.5,0.0,0.0,12.17,-12.4787092208862
2,1.75,3.50,4.00,0.30,1.25,1.50,1.5,4.5,0.0,0.0,12.26,-15.90647315979
3,1.75,3.50,4.00,0.30,1.25,1.50,1.5,4.5,0.0,0.0,12.35,-15.4567308425903
4,1.75,3.50,4.00,0.30,1.25,1.50,1.5,4.5,0.0,0.0,12.44,-14.8571949005127
...,...,...,...,...,...,...,...,...,...,...,...,...
11435,2.40,4.25,3.75,0.72,1.75,0.75,1.0,5.0,0.0,25.0,15.59,-14.8851690292358
11436,2.40,4.25,3.75,0.72,1.75,0.75,1.0,5.0,0.0,25.0,15.68,-15.9254865646362
11437,2.40,4.25,3.75,0.72,1.75,0.75,1.0,5.0,0.0,25.0,15.77,-14.6904439926147
11438,2.40,4.25,3.75,0.72,1.75,0.75,1.0,5.0,0.0,25.0,15.86,-14.9715461730957
